# Harnessing Machine Learning for Healthcare: Predicting Death Events of Patients with Prior Heart Failure

One of machine learning's most significant applications is diagnosing and predicting future outcomes of patients, given their demographics and medical histories. By harnessing the data to classify patients as "high risk", healthcare professionals can administer more specialized care and implement proactive measures to save the lives of many.

This project aims to analyze the characteristics of 299 patients who have experienced heart failure. The target variable of the dataset is a death event, denoting whether a patient died during the follow-up period. Patient features include demographics like age and sex, along with medical statistics like CPK level, platelet count, and etc. Following the exploratory data analysis (EDA), modeling will be performed using a multitude of different machine learning models to compare and achieve the highest accuracy. Lastly, a dashboard will be implemented to showcase key statistics, modeling outcomes, and predictions on input patient statistics.

<input type="checkbox"/><label for="checkbox"> EDA</label><br>
<input type="checkbox"/><label for="checkbox"> Modeling</label><br>
<input type="checkbox"/><label for="checkbox"> Dashboard</label><br>

In [36]:
__author__ = 'Jared Paul Guevara'

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

train = pd.read_csv('data/heart_failure_train.csv')
train.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
0,75.0,0,582,0,45,1,263358.03,1.18,137,1,0,87,0
1,65.0,0,113,1,25,0,497000.00,1.83,135,1,0,67,1
2,85.0,0,23,0,45,0,360000.00,3.00,132,1,0,28,1
3,70.0,1,171,0,60,1,176000.00,1.10,145,1,1,146,0
4,75.0,1,582,0,30,0,225000.00,1.83,134,1,0,113,1


In [38]:
X_train = train.iloc[:, :-1]
display(X_train.head())

y_train = train.iloc[:, -1]
y_train.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,45,1,263358.03,1.18,137,1,0,87
1,65.0,0,113,1,25,0,497000.00,1.83,135,1,0,67
2,85.0,0,23,0,45,0,360000.00,3.00,132,1,0,28
3,70.0,1,171,0,60,1,176000.00,1.10,145,1,1,146
4,75.0,1,582,0,30,0,225000.00,1.83,134,1,0,113


0    0
1    1
2    1
3    0
4    1
Name: death_event, dtype: int64

In [39]:
X_test = pd.read_csv('data/heart_failure_X_test.csv')
y_test = pd.read_csv('data/heart_failure_y_test.csv')

In [41]:
log_pipe = make_pipeline(StandardScaler(), LogisticRegression())
svc_pipe = make_pipeline(StandardScaler(), SVC())
rfc_pipe = make_pipeline(StandardScaler(), RandomForestClassifier())

model_names = {0 : "Logistic Regression",
               1 : "Support Vector Classifier",
               2 : "Random Forest Classifier"}

models = [log_pipe, svc_pipe, rfc_pipe]
preds = []
for model in models:
    model.fit(X_train, y_train)
    train_preds = model.predict(X_train)
    preds.append((round(accuracy_score(train_preds, y_train), 2), round(accuracy_score(test_preds, y_test), 2), round(cross_val_score(model, X_train, y_train, cv=10).mean(), 2)))

for i, pred in enumerate(preds):
    print(f'Accuracy metrics of {model_names[i]} model:\nTraining accuracy: {pred[0]}\nCross-validation accuracy across 10 folds: {pred[2]}\nTesting accuracy: {pred[1]}\n')

Accuracy metrics of Logistic Regression model:
Training accuracy: 0.87
Cross-validation accuracy across 10 folds: 0.83
Testing accuracy: 0.76

Accuracy metrics of Support Vector Classifier model:
Training accuracy: 0.92
Cross-validation accuracy across 10 folds: 0.8
Testing accuracy: 0.76

Accuracy metrics of Random Forest Classifier model:
Training accuracy: 1.0
Cross-validation accuracy across 10 folds: 0.86
Testing accuracy: 0.76



Considering that the Random Forest Classifier had the highest training and cross-validation accuracy, this will be the model used to predict patient outcomes, but not before performing some hyperparameter optimization. To do so, a randomized search of parameters will be performed using sklearn's `RandomizedSearchCV` function, which allows a range of inpute parameters to be searched through randomly, in order to identify the best performing values.

In [42]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(1, 1000),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': randint(1, 10),
    'min_samples_split': randint(1, 10),
    'min_samples_leaf': randint(2, 10),
    'max_leaf_nodes': randint(1, 100),
    'max_features': ['sqrt', 'log2', 12, 6],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

rfc = RandomForestClassifier()

rand_search = RandomizedSearchCV(rfc, param_distributions=param_dist, scoring='accuracy', cv=10, random_state=42)
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(),
                   param_distributions={'class_weight': [None, 'balanced',
                                                         'balanced_subsample'],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f9316c29030>,
                                        'max_features': ['sqrt', 'log2', 12, 6],
                                        'max_leaf_nodes': <scipy.stats._distn_infrastruct...rozen object at 0x7f9316b13580>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f9316c29840>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f9316b137f0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f931fa67cd0>},
                   random_state=42, scoring='accuracy')

In [ ]:
optimal_params = rand_search.best_params_
optimal_pipe = make_pipeline(StandardScaler(), RandomForestClassifier(**optimal_params))

optimal_pipe.fit(X_train, y_train)
train_preds = model.predict(X_train)